In [20]:
import pandas as pd
import pymssql
import pandas.io.sql as pdsql
query = \
"""
SELECT corpName, productName, productCode FROM productCode
"""
conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", "intake")
df_code = pdsql.read_sql_query(query, con=conn)
conn.close()

In [21]:
df_code.head()

,corpName,productName,productCode
0,intake,밀스드링크,11100
1,intake,밀스,11200
2,intake,밀스3.1그레인보틀형,11201
3,intake,밀스3.1코코넛보틀형,11202
4,intake,밀스3.소이파우치형,11203


In [22]:
df = pd.read_csv('SSG_CMT.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head(3)

,usr,dte,rte,cmt,itm,opt
0,hub*******,2018.07.11,100%,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ,(3+랜덤증정)간편한 아침 모닝죽 1주일(130gx7팩) 단호박/단팥/귀리/검은콩,[구매옵션] 맛선택:검은콩 / 수량:7개입
1,hub*******,2018.07.11,100%,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ,(3+랜덤증정)간편한 아침 모닝죽 1주일(130gx7팩) 단호박/단팥/귀리/검은콩,[구매옵션] 맛선택:귀리 / 수량:7개입
2,hub*******,2018.07.11,100%,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ,(3+랜덤증정)간편한 아침 모닝죽 1주일(130gx7팩) 단호박/단팥/귀리/검은콩,[구매옵션] 맛선택:단호박 / 수량:7개입


In [23]:
df_upload = pd.DataFrame(columns = ['mall', 'corpName', 'productCode', 'date', 'id', 'productScore', 'recommScore', 'main_text'])
df_upload['productCode'] = df['itm'] + df['opt'].fillna('')
df_upload['date'] = df.dte
df_upload['id'] = df.usr
df_upload['productScore'] = df.rte 
df_upload['recommScore'] = 0
df_upload['main_text'] = df.cmt
df_upload['mall'] = 'ssg'
df_upload['corpName'] = 'intake'

df_upload.head(3)

,mall,corpName,productCode,date,id,productScore,recommScore,main_text
0,ssg,intake,(3+랜덤증정)간편한 아침 모닝죽 1주일(130gx7팩) 단호박/단팥/귀리/검은콩[...,2018.07.11,hub*******,100%,0,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ
1,ssg,intake,(3+랜덤증정)간편한 아침 모닝죽 1주일(130gx7팩) 단호박/단팥/귀리/검은콩[...,2018.07.11,hub*******,100%,0,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ
2,ssg,intake,(3+랜덤증정)간편한 아침 모닝죽 1주일(130gx7팩) 단호박/단팥/귀리/검은콩[...,2018.07.11,hub*******,100%,0,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ


In [24]:
df_word = pd.read_csv('SSG_WORD_CSV.csv', encoding='cp949')
df_word['code'] = df_word.new

In [25]:
mapping = dict()
for name, code in df_code[['productName','productCode']].as_matrix():
    mapping[name] = code

df_word_ex = df_word[df_word.new_word == 0]
df_word_ex['code'] = df_word_ex['code'].replace(mapping)

df_word_nw = df_word[df_word.new_word == 1]

C:\Users\lion5\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
mapping2 = dict()
for name, code in df_word_ex[['original','code']].as_matrix():
    mapping2[name] = code
df_upload['productCode'] = df_upload.productCode.replace(mapping2)

df_upload.drop(df_upload[df_upload.productCode.apply(type) != int].index, inplace = True)
df_upload['productCode'] = df_upload.productCode.apply(int)

df_upload['productScore'] = df_upload.productScore.apply(lambda x: x.replace('%',''))
df_upload['productScore'] = df_upload.productScore.apply(int) // 20

df_upload['date'] = pd.to_datetime(df_upload['date'])

In [27]:
df_upload.head()

,mall,corpName,productCode,date,id,productScore,recommScore,main_text
0,ssg,intake,12111,2018-07-11,hub*******,5,0,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ
1,ssg,intake,12113,2018-07-11,hub*******,5,0,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ
2,ssg,intake,12109,2018-07-11,hub*******,5,0,간편하게 먹기 너무 좋아요\r\n한번에 6박스 샀어요 ㅎ
3,ssg,intake,12109,2018-07-11,hea*******,5,0,식사대용으로 구입했습니다\r\n괜찮아요
4,ssg,intake,12113,2018-07-11,set*******,5,0,예상보다 든든하고 맛도 괜찮아서 마음에 들어요.


In [28]:
df_upload.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1150 entries, 0 to 1155
Data columns (total 8 columns):
mall            1150 non-null object
corpName        1150 non-null object
productCode     1150 non-null int64
date            1150 non-null datetime64[ns]
id              1150 non-null object
productScore    1150 non-null int64
recommScore     1150 non-null int64
main_text       1150 non-null object
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 80.9+ KB


In [29]:
conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", "intake")
cursor = conn.cursor()

data = df_upload.values.tolist()
data = [tuple(lst) for lst in data]

print('executemany 시작')
cursor.executemany("insert into productReview (mall, corpName, productCode, date, id, productScore, recommScore, main_text) values(%s, %s, %d, %s, %s, %d, %d, %s)", data)

print('commit 시작')
conn.commit()

conn.close()